In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler

# Set random seed for PyTorch and NumPy
seed = 14618
torch.manual_seed(seed)
np.random.seed(seed)

# Load your data from mydata_petals.csv
data = pd.read_csv('mydata_petals.csv')

# Extract the last column (EAB) as the target variable
EAB = data.iloc[:, -1].values

# Extract the first 10 columns as input features
input = data.iloc[:, :-1].values

# Create MinMaxScaler objects for input and EAB
scaler_in = MinMaxScaler(feature_range=(-0.5, 0.5))
scaler_out = MinMaxScaler(feature_range=(-0.99, 0.99))

# Fit and transform input data, and store the transformation matrix ps_in
p_scaled = scaler_in.fit_transform(input)
p = torch.FloatTensor(p_scaled.T)
ps_in = scaler_in.scale_

# Fit and transform EAB data, and store the transformation matrix ts_out
t_scaled = scaler_out.fit_transform(EAB.reshape(-1, 1))
t = torch.FloatTensor(t_scaled)
ts_out = scaler_out.scale_

print("Scaled Data Shapes:")
print(p_scaled.shape)
print(t_scaled.shape)

# Define the ratio for training and testing data
train_ratio = 0.80  # 80% training data, 
test_ratio = 0.20   # 20% testing data

# Number of samples
n_samples = len(EAB)

# Create a random permutation of indices for shuffling
indices = np.random.permutation(n_samples)

# Calculate the sizes of the training and test sets
n_train = int(train_ratio * n_samples)
n_test = n_samples - n_train

# Use the shuffled indices to split the data into training and test sets
train_indices = indices[:n_train]
test_indices = indices[n_train:]

# Split the data into training and test sets based on the shuffled indices
X_train, y_train = p_scaled[train_indices], t_scaled[train_indices]
X_test, y_test = p_scaled[test_indices], t_scaled[test_indices]

print("Training Data Shapes:")
print(X_train.shape, y_train.shape)
print("Test Data Shapes:")
print(X_test.shape, y_test.shape)

# 打印训练数据前五行
print("First 5 rows of Training Data:")
print(X_train[:5])
print("First 5 rows of Training Target (y_train):")
print(y_train[:5])

print("Test Data Shapes:")
print(X_test.shape, y_test.shape)

# 打印测试数据前五行
print("First 5 rows of Test Data:")
print(X_test[:5])
print("First 5 rows of Test Target (y_test):")
print(y_test[:5])

Scaled Data Shapes:
(2000, 10)
(2000, 1)
Training Data Shapes:
(1600, 10) (1600, 1)
Test Data Shapes:
(400, 10) (400, 1)
First 5 rows of Training Data:
[[ 0.38088088  0.20570571 -0.07357357 -0.32865731 -0.29468116  0.302
   0.37907376  0.17089453  0.37363897 -0.4012016 ]
 [ 0.37887888  0.34084084  0.02352352  0.21342685 -0.41792965  0.17933333
   0.2058319   0.36782377 -0.40773639 -0.02136182]
 [-0.35185185 -0.12662663  0.48698699  0.37274549  0.26694309  0.242
  -0.13150372  0.2329773   0.21977077  0.08210948]
 [ 0.26176176 -0.17167167  0.45695696  0.29158317  0.00757792 -0.46466667
   0.39736993 -0.4506008   0.16475645 -0.25367156]
 [-0.36586587 -0.16566567  0.23473473  0.0511022   0.00729196  0.03066667
   0.43110349 -0.11014686 -0.17421203 -0.3164219 ]]
First 5 rows of Training Target (y_train):
[[-0.1889313 ]
 [-0.2040458 ]
 [ 0.32496183]
 [ 0.03778626]
 [-0.00755725]]
Test Data Shapes:
(400, 10) (400, 1)
First 5 rows of Test Data:
[[ 0.36586587  0.36086086  0.0035035   0.21042084

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.model_selection import cross_val_score
from skopt import BayesSearchCV

# Define the architecture of the DNN with hidden layers
class DNN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dims):
        super(DNN, self).__init__()
        dims = [input_dim] + hidden_dims + [output_dim]
        self.layers = nn.ModuleList()
        for i in range(1, len(dims)):
            self.layers.append(nn.Linear(dims[i - 1], dims[i]))
            if i < len(dims) - 1:
                self.layers.append(nn.Tanh())

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

# Wrapper class to make PyTorch model compatible with scikit-optimize
class PyTorchWrapper(BaseEstimator):
    def __init__(self, model_class, input_dim, output_dim, hidden_dims, learning_rate=0.01, max_epochs=3000):
        self.model_class = model_class
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dims = hidden_dims
        self.learning_rate = learning_rate
        self.max_epochs = max_epochs
        self.model = None

    def fit(self, X, y):
        self.model = self.model_class(self.input_dim, self.output_dim, self.hidden_dims)
        inputs = torch.FloatTensor(X)
        targets = torch.FloatTensor(y)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        
        for epoch in range(self.max_epochs):
            optimizer.zero_grad()
            outputs = self.model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

    def predict(self, X):
        inputs = torch.FloatTensor(X)
        outputs = self.model(inputs)
        return outputs.detach().numpy()

    def score(self, X, y):
        predictions = self.predict(X)
        mse = np.mean((predictions - y) ** 2)
        return -mse  # Negate for minimization

    def set_params(self, **params):
        for param, value in params.items():
            setattr(self, param, value)
        return self

    def get_params(self, deep=True):
        return {
            'model_class': self.model_class,
            'input_dim': self.input_dim,
            'output_dim': self.output_dim,
            'hidden_dims': self.hidden_dims,
            'learning_rate': self.learning_rate,
            'max_epochs': self.max_epochs,
        }

# Objective function to be used by scikit-optimize
def objective(params):
    # Extract hyperparameters
    learning_rate = params['learning_rate']
    max_epochs = int(params['max_epochs'])

    # Create PyTorch wrapper
    model_params = {
        'input_dim': 10,
        'output_dim': 1,
        'hidden_dims': [
            int(params['hidden_dim_1']),
            int(params['hidden_dim_2']),
            int(params['hidden_dim_3']),
            int(params['hidden_dim_4'])
        ],
        'learning_rate': learning_rate,
        'max_epochs': max_epochs
    }
    model = PyTorchWrapper(DNN, **model_params)

    # Calculate performance metric (negative mean squared error for minimization)
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring='neg_mean_squared_error')
    avg_score = -np.mean(scores)

    return avg_score

# Define search space
search_spaces = {
    'hidden_dim_1': (8, 16),
    'hidden_dim_2': (8, 16),
    'hidden_dim_3': (4, 10),
    'hidden_dim_4': (2, 8),
    'learning_rate': (1e-4, 1e-1, 'log-uniform'),
    'max_epochs': (100, 3000),
}

# Create BayesSearchCV object
opt = BayesSearchCV(
    PyTorchWrapper(model_class=DNN, input_dim=10, output_dim=1, hidden_dims=[8, 8, 4, 2]),
    search_spaces,
    n_iter=30,
    n_jobs=-1,
    cv=3,
    random_state=42
)

# Run optimization process
opt.fit(np.array(X_train), np.array(y_train))

# Output the best parameters
print("Best parameters found: ", opt.best_params_)


Best parameters found:  OrderedDict([('hidden_dim_1', 8), ('hidden_dim_2', 16), ('hidden_dim_3', 4), ('hidden_dim_4', 5), ('learning_rate', 0.00628528834895659), ('max_epochs', 2738)])
